In [2]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    # Connecting sql command to variable 
    film = conn.sql("FROM film;").df()
    category = conn.sql("FROM category;").df()
    film_category = conn.sql("FROM film_category;").df()
    language = conn.sql("FROM language;").df()
    actor = conn.sql("FROM actor;").df()
    film_actor = conn.sql("FROM film_actor;").df()
    staff = conn.sql("FROM staff;").df()
    inventory = conn.sql("FROM inventory;").df()
    payment = conn.sql("FROM payment;").df()
    customer = conn.sql("FROM customer;").df()
    rental = conn.sql("FROM rental;").df()
    country = conn.sql("FROM country;").df()
    city = conn.sql("FROM city;").df()
    address = conn.sql("FROM address;").df()
    store = conn.sql("FROM store;").df()
    description = conn.sql("DESC;").df()




## Task 1 - EDA in python

In [3]:
# a) Movies longer than 3 hours

duckdb.sql("""
    SELECT title, length as length_in_minutes
    FROM film
    WHERE length > 180
    ORDER BY length DESC;
""").df()

,title,length_in_minutes
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


In [4]:
# b) Movies with the word "love" in the title

duckdb.sql("""
    SELECT title, rating, length, description
    FROM film
    WHERE title ILIKE '%love%';
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


In [5]:
# c) Length statistics

duckdb.sql("""
    SELECT
    MIN(length) AS Shortest_movie_length,
    ROUND(AVG(length)) AS Average_movie_length,
    MEDIAN(length) AS Median_movie_lenth,
    MAX(length) AS Longest_movie_length
    FROM film;
""").df()

,Shortest_movie_length,Average_movie_length,Median_movie_lenth,Longest_movie_length
0,46,115.0,114.0,185


In [27]:
# d) Most expensive movies to rent per day

duckdb.sql(
    """
    SELECT title, 
    rental_rate, 
    rental_duration,
    (rental_rate/rental_duration) AS rental_cost_per_day
    FROM film
    ORDER BY rental_cost_per_day DESC
    LIMIT 10;
"""
).df()

,title,rental_rate,rental_duration,rental_cost_per_day
0,AMERICAN CIRCUS,4.99,3,1.663333
1,BACKLASH UNDEFEATED,4.99,3,1.663333
2,BILKO ANONYMOUS,4.99,3,1.663333
3,BEAST HUNCHBACK,4.99,3,1.663333
4,CARIBBEAN LIBERTY,4.99,3,1.663333
5,AUTUMN CROW,4.99,3,1.663333
6,CASPER DRAGONFLY,4.99,3,1.663333
7,ACE GOLDFINGER,4.99,3,1.663333
8,BEHAVIOR RUNAWAY,4.99,3,1.663333
9,CASUALTIES ENCINO,4.99,3,1.663333


In [ ]:
# Check if the result above is reasonable 
duckdb.sql("""
    SELECT
    rental_rate, rental_duration
    FROM film
    ORDER BY rental_rate desc;
""").df()

,rental_rate,rental_duration
0,4.99,6
1,4.99,3
2,4.99,4
3,4.99,6
4,4.99,7
...,...,...
995,0.99,3
996,0.99,6
997,0.99,4
998,0.99,6


In [ ]:
# e) Top 10 actors whos played in most movies
# I count the number of 'film_id' to get number of films,
# and then I join table 'actor' with 'film_actor' by actor_id,
# and table 'film_actor' with 'film' by film_id.

duckdb.sql("""
    SELECT
        a.first_name || ' ' || a.last_name as actor,
        COUNT(f.film_id) as number_films
    FROM actor a
    LEFT JOIN film_actor fa ON fa.actor_id = a.actor_id 
    LEFT JOIN film f ON f.film_id = fa.film_id
    GROUP BY actor
    ORDER BY number_films DESC
    LIMIT 10;
""").df()

,actor,number_films
0,SUSAN DAVIS,54
1,GINA DEGENERES,42
2,WALTER TORN,41
3,MARY KEITEL,40
4,MATTHEW CARREY,39
5,SANDRA KILMER,37
6,SCARLETT DAMON,36
7,VAL BOLGER,35
8,HENRY BERRY,35
9,ANGELA WITHERSPOON,35


### f) My own questions to explore the sakila database

In [ ]:
# 1. Which is the top 5 most rented film categorys among customers?
# Caluculate number of rentals
# JOIN tables to get the number of rented films
duckdb.sql("""
    SELECT
        c.name AS category,
        COUNT(r.rental_id) AS total_rentals
    FROM category c
    INNER JOIN film_category fc ON fc.category_id = c.category_id 
    INNER JOIN film f ON f.film_id = fc.film_id
    INNER JOIN inventory i ON i.film_id = f.film_id
    INNER JOIN rental r ON r.inventory_id = i.inventory_id
    GROUP BY category
    ORDER BY total_rentals DESC
    LIMIT 5;
""").df()

,category,total_rentals
0,Sports,1179
1,Animation,1166
2,Action,1112
3,Sci-Fi,1101
4,Family,1096


In [ ]:
# 2. 'Susan Davis' has played in most movies, 
# but in what top 3 genre has she done most movies?
duckdb.sql("""
    SELECT
        a.first_name || ' ' || a.last_name as actor,
        c.name as top_category,
        COUNT(fa.film_id) AS films_per_category
    FROM actor a
    INNER JOIN film_actor fa ON fa.actor_id = a.actor_id 
    INNER JOIN film_category fc ON fa.film_id = fc.film_id
    INNER JOIN category c ON c.category_id = fc.category_id 
    WHERE a.first_name ILIKE 'Susan' AND a.last_name ILIKE 'Davis'
    GROUP BY actor, top_category
    ORDER BY films_per_category DESC
    LIMIT 3;
""").df()

,actor,top_category,films_per_category
0,SUSAN DAVIS,Children,6
1,SUSAN DAVIS,Action,5
2,SUSAN DAVIS,Documentary,4


In [55]:
language

,language_id,name,last_update
0,1,English,2021-03-06 15:51:48
1,2,Italian,2021-03-06 15:51:48
2,3,Japanese,2021-03-06 15:51:48
3,4,Mandarin,2021-03-06 15:51:48
4,5,French,2021-03-06 15:51:48
5,6,German,2021-03-06 15:51:48


In [69]:
# 3. Which movies has language 'Mandarin'?
duckdb.sql("""
    SELECT
        l.name as language,
        f.title
    FROM language l
    INNER JOIN film f ON l.language_id = f.language_id
    WHERE l.name = 'Mandarin';
""").df()

,language,title


In [73]:
# 4. Are there any movies with another language than 'English'?
duckdb.sql("""
    SELECT
        l.name as language,
        COUNT(f.film_id) AS number_of_films
    FROM language l
    INNER JOIN film f ON l.language_id = f.language_id
    GROUP BY language
    ORDER BY number_of_films DESC;
""").df()

,language,number_of_films
0,English,1000


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,<NA>,6,0.99,183,9.99,G,"Trailers,Behind the Scenes",2021-03-06 15:52:08
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,<NA>,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes",2021-03-06 15:52:08
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,<NA>,6,0.99,105,10.99,NC-17,Deleted Scenes,2021-03-06 15:52:08
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,<NA>,5,2.99,101,28.99,R,"Trailers,Deleted Scenes",2021-03-06 15:52:08
